## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
import numpy as np
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# 讀取乳癌資料集
breast_cancer = datasets.load_breast_cancer()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(breast_cancer.data, breast_cancer.target, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingRegressor(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果，約為 0.03 的 MSE
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

0.03034575209701989


In [10]:
# 設定要訓練的超參數組合
n_estimators = [10, 50, 100, 200, 300]
max_depth = [1, 3, 5]
learning_rate = [0.05, 0.1, 0.3]
min_samples_leaf = [3, 5, 9, 15]
# max_features = [1.0, 0.3, 0.1]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, 
                 min_samples_leaf=min_samples_leaf) 
                 # max_features=max_features)


## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
#grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=3, pre_dispatch='n_jobs', verbose=1, cv=3)
# in my Mac Pro, n_jobs>1 will raise memory errors
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=1, # my Mac Pro can only n_jobs=1
                           verbose=1, cv=5, 
                           iid=False, error_score=np.nan )


# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 900 out of 900 | elapsed:  1.3min finished


In [11]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.034254 using {'learning_rate': 0.3, 'max_depth': 5, 'min_samples_leaf': 9, 'n_estimators': 50}


In [12]:
grid_result.best_params_

{'learning_rate': 0.3,
 'max_depth': 5,
 'min_samples_leaf': 9,
 'n_estimators': 50}

In [13]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'],
                                         learning_rate=grid_result.best_params_['learning_rate'], 
                                          min_samples_leaf=grid_result.best_params_['min_samples_leaf'])
#                                          max_features=grid_result.best_params_['max_features'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [14]:
# 調整參數後約可降至 0.035 的 MSE !!!
print(metrics.mean_squared_error(y_test, y_pred))

0.03508919607708003
